In [2]:
import pandas as pd
import os
import glob
import plotly
import dash
from posicionamento import posicionamento_partidos

In [3]:
all_files = glob.glob(os.path.join("dados/*.csv"))
all_files
li = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0, sep=";")
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df = df.drop(["uriVotacao", "deputado_uri", "deputado_uriPartido", "deputado_urlFoto", "deputado_idLegislatura"], axis=1)

df_nan = df[df["voto"].isnull()].groupby("idVotacao").count()
df = df[~df["idVotacao"].isin(df_nan.index)]

df["voto"].replace({"Sim": 1, "Não": -1, "Abstenção": 0, 'Artigo 17':0, 'Obstrução':0, 'Favorável com restrições': 0, 'Branco': 0 }, inplace=True)
df["voto_count"] = 1
df["dataHoraVoto"] = pd.to_datetime(df['dataHoraVoto'])
df["ano"] = pd.DatetimeIndex(df['dataHoraVoto']).year

In [4]:
n_df = df.copy()

In [5]:
n_df["posicionamento"] = n_df["deputado_siglaPartido"]
n_df["posicionamento"].replace(posicionamento_partidos, inplace=True)
n_df

,idVotacao,dataHoraVoto,voto,deputado_id,deputado_nome,deputado_siglaPartido,deputado_siglaUf,voto_count,ano,posicionamento
0,37897-3,2001-12-11 19:49:15,1,73886,Adão Pretto,PT,RS,1,2001,Esquerda
1,37897-3,2001-12-11 19:29:35,1,74328,Adauto Pereira,PFL,PB,1,2001,Centro-direita
2,37897-3,2001-12-11 19:26:34,1,74450,Adolfo Marinho,PSDB,CE,1,2001,Centro
3,37897-3,2001-12-11 19:25:14,0,74646,Aécio Neves,PSDB,MG,1,2001,Centro
4,37897-3,2001-12-11 19:26:59,1,73765,Affonso Camargo,PFL,PR,1,2001,Centro-direita
...,...,...,...,...,...,...,...,...,...,...
1329477,945554-186,2022-04-07 13:21:12,1,178889,Zé Carlos,PT,MA,1,2022,Esquerda
1329478,945554-186,2022-04-07 13:22:05,1,204559,Zé Neto,PT,BA,1,2022,Esquerda
1329479,945554-186,2022-04-07 13:26:44,-1,160632,Zé Silva,SOLIDARIEDADE,MG,1,2022,Centro
1329480,945554-186,2022-04-07 13:21:07,-1,204517,Zé Vitor,PL,MG,1,2022,Centro-direita


In [6]:
n_df["voto"].unique()

array([ 1,  0, -1], dtype=int64)

In [7]:
pos_votos = n_df.groupby(["posicionamento", "voto"], as_index=False).count().drop(["idVotacao", "ano", "dataHoraVoto", "deputado_id", "deputado_nome", "deputado_siglaPartido", "deputado_siglaUf"], axis=1)
pos_votos["voto"].replace({-1: "Não", 0: "Abstenção", 1: "Sim"}, inplace=True)

pos_votos_total = pos_votos.groupby("posicionamento").sum()
votos_totais = [v for v in pos_votos_total.voto_count.tolist() for i in range(3)]
pos_votos.insert(3, "total", votos_totais)

pos_votos["porcentagem"] = pos_votos["voto_count"] / pos_votos["total"]
pos_votos


,posicionamento,voto,voto_count,total,porcentagem
0,Centro,Não,177283,406344,0.436288
1,Centro,Abstenção,11807,406344,0.029057
2,Centro,Sim,217254,406344,0.534655
3,Centro-direita,Não,98261,223307,0.440027
4,Centro-direita,Abstenção,9115,223307,0.040818
5,Centro-direita,Sim,115931,223307,0.519155
6,Centro-esquerda,Não,69139,162045,0.426665
7,Centro-esquerda,Abstenção,7908,162045,0.048801
8,Centro-esquerda,Sim,84998,162045,0.524533
9,Direita,Não,146730,315098,0.465665


In [8]:
import plotly.express as px
fig = px.bar(
    pos_votos.query("posicionamento != 'Sem partido'"), 
    x="posicionamento",
    color="voto",
    color_discrete_map={
        "Sim": 'rgb(0, 122, 255)',
        "Não": 'rgb(220, 20, 60)',
        "Abstenção": 'rgb(236, 164, 54)'
    },

    y="porcentagem"
)
fig.show()

In [9]:
fig = px.line(
    pos_votos.query("posicionamento != 'Sem partido'"),
    title="Posicionamento por ideologia",
    x="posicionamento",
    y="porcentagem",
    color="voto",
    color_discrete_map={
        "Sim": 'rgb(99, 110, 250)',
        "Não": 'rgb(239, 85, 59)',
        "Abstenção": 'rgb(125, 147, 152)'
    },
    barmode="group",
)
fig.show()

In [16]:
estados_df = df.copy().drop(["deputado_id", "deputado_nome", "dataHoraVoto", "idVotacao"], axis=1)
estados_df["voto"].replace({-1: "Não", 0: "Abstenção", 1: "Sim"}, inplace=True)

# estados_df_2003_2006 = estados_df.query("ano >= 2003 and ano <= 2006").groupby(["deputado_siglaUf", "voto"], as_index=False).sum().drop(["ano"], axis=1)
# estados_df_2007_2010 = estados_df.query("ano >= 2007 and ano <= 2010").groupby(["deputado_siglaUf", "voto"], as_index=False).sum().drop(["ano"], axis=1)
# estados_df_2011_2014 = estados_df.query("ano >= 2011 and ano <= 2014").groupby(["deputado_siglaUf", "voto"], as_index=False).sum().drop(["ano"], axis=1)
# estados_df_2015_2018 = estados_df.query("ano >= 2015 and ano <= 2018").groupby(["deputado_siglaUf", "voto"], as_index=False).sum().drop(["ano"], axis=1)
# estados_df_2019_2022 = estados_df.query("ano >= 2019 and ano <= 2022").groupby(["deputado_siglaUf", "voto"], as_index=False).sum().drop(["ano"], axis=1)

In [17]:
estados_df = df.copy().drop(["deputado_id", "deputado_nome", "dataHoraVoto", "idVotacao"], axis=1)
estados_df["voto"].replace({-1: "Não", 0: "Abstenção", 1: "Sim"}, inplace=True)

estados_df = estados_df.groupby(["deputado_siglaUf", "voto"], as_index=False).count().drop(["ano", "deputado_siglaPartido"], axis=1)

pos_votos_totais = estados_df.groupby("deputado_siglaUf").sum()
votos_totais = [v for v in pos_votos_totais.voto_count.tolist() for i in range(3)]

estados_df
estados_df.insert(3, "total", votos_totais)

estados_df["porcentagem"] = estados_df["voto_count"] / estados_df["total"]
estados_df

,deputado_siglaUf,voto,voto_count,total,porcentagem
0,AC,Abstenção,948,21152,0.044818
1,AC,Não,9239,21152,0.436791
2,AC,Sim,10965,21152,0.518391
3,AL,Abstenção,1260,21433,0.058788
4,AL,Não,9060,21433,0.422713
...,...,...,...,...,...
76,SP,Não,82083,184978,0.443745
77,SP,Sim,93905,184978,0.507655
78,TO,Abstenção,549,21013,0.026127
79,TO,Não,9196,21013,0.437634


In [18]:
fig = px.histogram(
    estados_df,
    title="Votacao por estado",
    x="deputado_siglaUf",
    y="porcentagem",
    color="voto",
    color_discrete_map={
        "Sim": 'rgb(99, 110, 250)',
        "Não": 'rgb(239, 85, 59)',
        "Abstenção": 'rgb(125, 147, 152)'
    },
    barmode="group",
)
fig.show()

In [19]:
fig = px.histogram(
    estados_df,
    title="Votacao por estado",
    x="deputado_siglaUf",
    y="voto_count",
    color="voto",
    color_discrete_map={
        "Sim": 'rgb(99, 110, 250)',
        "Não": 'rgb(239, 85, 59)',
        "Abstenção": 'rgb(125, 147, 152)'
    },
    barmode="group",
)

fig.show()